<a href="https://colab.research.google.com/github/vipashaaV321/Collaborative-Food-Recipe-Recommendation-System/blob/Vipasha/ITEM_ITEM_SVD3_Vipasha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Base_path ='/content/drive/MyDrive/Data/RS2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [ ]:
data = pd.read_csv(Base_path+"/interactions_train.csv")

In [ ]:
data

In [ ]:
subset1 = data[:349450] 
subset2 = data[349451:]

In [ ]:
matrix1 = subset1.pivot_table(values='rating', index='user_id', columns='recipe_id')
matrix2 = subset2.pivot_table(values='rating', index='user_id', columns='recipe_id')

In [ ]:
complete_matrix = matrix1.join(matrix2)

In [ ]:
data.groupby(['user_id','recipe_id'])['rating'].max().unstack()

In [ ]:
# Decompose the user-item ratings matrix using SVD
U, sigma, Vt = svds(data, k=50)
sigma = np.diag(sigma)

In [ ]:
U

In [ ]:
sigma

In [ ]:
Vt

In [ ]:
# Predict the ratings for all users and items
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [ ]:
predicted_ratings

In [ ]:
# Define the number of recommended items
num_recommendations = 10

# Calculate the item-item similarity matrix using cosine similarity
item_similarities = cosine_similarity(predicted_ratings.T)

In [ ]:
# Recommend top items based on similarity for each user
top_items = np.zeros((predicted_ratings.shape[0], num_recommendations))
for i in range(predicted_ratings.shape[0]):
    top_items[i] = np.argsort(-item_similarities[i])[:num_recommendations]

In [ ]:
# Define the ground truth ratings and item indices
ground_truth_ratings = data.values
item_indices = np.arange(ground_truth_ratings.shape[1])

In [ ]:
# Calculate the precision, recall, and F1-score for each user
precision = 0.0
recall = 0.0
f1_score = 0.0
for i in range(ground_truth_ratings.shape[0]):
    relevant_items = np.where(ground_truth_ratings[i] > 0)[0]
    recommended_items = top_items[i].astype(int)
    tp = len(set(recommended_items) & set(relevant_items))
    fp = len(set(recommended_items) - set(relevant_items))
    fn = len(set(relevant_items) - set(recommended_items))
    precision += tp / (tp + fp + 1e-9)
    recall += tp / (tp + fn + 1e-9)
    f1_score += 2 * tp / (2 * tp + fp + fn + 1e-9)
precision /= ground_truth_ratings.shape[0]
recall /= ground_truth_ratings.shape[0]
f1_score /= ground_truth_ratings.shape[0]
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)

In [ ]:
# Calculate the NDCG and MRR for each user
dcg = 0.0
mrr = 0.0
for i in range(ground_truth_ratings.shape[0]):
    relevant_items = np.where(ground_truth_ratings[i] > 0)[0]
    recommended_items = top_items[i].astype(int)
    tp = len(set(recommended_items) & set(relevant_items))
    if tp > 0:
        # Calculate NDCG
        idcg = np.sum(1 / np.log2(np.arange(2, tp + 2)))
        dcg += np.sum(1 / np.log2(np.where(np.isin(item_indices, recommended_items[:tp]))[0] + 2)) / idcg
        # Calculate MRR
        mrr += 1 / (np.min(np.where(item_indices == recommended_items[0])[0]) + 1)
dcg /= ground_truth_ratings.shape 